In [3]:
import requests
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

## OMNI

### Scrape Omni restaurants

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.google.com/maps/d/u/0/viewer?mid=1qN2EGFpFtGzVtUVAwiP74IFJPMX59pu2&ll=33.72331810000003%2C-117.789675&z=8'
driver.get(url)
data = []
# wait for options and select restaurants
restaurants = WebDriverWait(driver,5).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, "HzV7m-pbTTYe-SmKAyb-haAclf")))[-1]
#click dropdown menu and find all restaurant links
restaurants.find_element(By.CLASS_NAME, 'HzV7m-pbTTYe-KoToPc-ornU0b').click()
time.sleep(1)
restaurants = restaurants.find_elements(By.CLASS_NAME, 'HzV7m-pbTTYe-ibnC6b-V67aGc')
print(f'found {len(restaurants)} restaurants')
# for each restaurant, save name, click to reveal address and save address
for r in restaurants[1:]:
    name = r.text
    r.click()
    try:
        address = WebDriverWait(driver,1).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, "fO2voc-jRmmHf-MZArnb-Q7Zjwb")))[0].text
    except:
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        continue
    data.append([name,address])

    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
# Close the webdriver
driver.quit()

found 63 restaurants


In [ ]:
omni_df = pd.DataFrame(data, columns=['name','address'])
omni_df

,name,address
0,Veg'n Out,"4606 Mission Gorge Pl, San Diego, CA 92120"
1,Kenshō,"16511 Magnolia St, Westminster, CA 92683"
2,Goldilocks Filipino Cuisine (SFO T1),San Francisco International Airport-T1 Harvey ...
3,Veggie Fam,"1500 Griffith Park Blvd, Los Angeles, CA 90189"
4,Kau Kau,"855 57th St Suite C, Sacramento, CA 95819"
5,Lee's Sandwiches,"2471 Berryessa Rd #3, San Jose, CA 95133"
6,Lee's Sandwiches,"260 E Santa Clara St, San Jose, CA 95113"
7,Yu Seafood Yorkdale,"3401 Dufferin St Unit 601A, Toronto, ON M6A 2T..."
8,Bring Me Some,"638 Concession St, Hamilton, ON L8V 1B5, Canada"
9,MATHILDA'S - Plant-based eating at its best,"29 Simcoe St S, Oshawa, ON L1H 4G1, Canada"


### Get place Ids

In [ ]:
#key api key
with open('/Users/alexanderdaffara/Documents/api_info/ImpactRestaurantSearch/Impact_maps_api_key.txt', 'r') as f:
    key = f.readline()
pids = []
# Iterate over the restaurants and retrieve their Place IDs
for name, address in zip(omni_df.name, omni_df.address):
    # URL for Place Search API
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={name} {address}&inputtype=textquery&fields=place_id&key={key}'

    # Send a GET request to the API endpoint
    response = requests.get(url)

    # Check if the response was successful (HTTP status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()

        # Extract the Place ID from the response
        if data['status'] == 'OK':
            place_id = data['candidates'][0]['place_id']
            pids.append(place_id)
        else:
            print(f'Request for {name} succeeded with status {data["status"]}')
            pids.append(np.nan)
    else:
        # Print an error message if the response was unsuccessful
        print(f'Request for {name} failed with error code {response.status_code}')
        pids.append(np.nan)
omni_df['pid'] = pids

Request for Lee's Sandwiches succeeded with status REQUEST_DENIED
Request for Scramblers Cafe succeeded with status REQUEST_DENIED
Request for L&L Hawaiian Barbecue succeeded with status REQUEST_DENIED
Request for L&L Hawaiian Barbecue succeeded with status REQUEST_DENIED
Request for L&L Hawaiian Barbecue succeeded with status REQUEST_DENIED


### Get place details

In [ ]:
#key api key
with open('/Users/alexanderdaffara/Documents/api_info/ImpactRestaurantSearch/Impact_maps_api_key.txt', 'r') as f:
    key = f.readline()

place_details = reset_place_details()

# Iterate over the restaurants and retrieve their Place IDs
for name, pid in zip(omni_df.name, omni_df.pid):
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={pid}&key={key}"

    response = requests.request("GET", url)

    # Check if the response was successful (HTTP status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()

        # Extract the Place ID from the response
        if data['status'] == 'OK':
            place_details = update_place_details(place_details, data['result'])
        else:
            print(f'Request for {name}, {pid} succeeded with status {data["status"]}')
            place_details = update_place_details(place_details, None)
    else:
        # Print an error message if the response was unsuccessful
        print(f'Request for {name}, {pid} failed with error code {response.status_code}')
        place_details = update_place_details(place_details, None)

#### merge Scraping df and place details df

In [ ]:
merged_df = omni_df.merge(pd.DataFrame(place_details, columns=place_details.keys()).drop(columns=['name']),
                          how='inner',
                          left_on='pid',
                          right_on='place_id',
                          suffixes=['_omni','_google'])
merged_df.rename(columns={'opening_hours/weekday_text':'opening_hours', 'editorial_summary/overview': 'editorial_summary'}, inplace=True)
merged_df


,name,address,pid,place_id,business_status,curbside_pickup,opening_hours,delivery,dine_in,editorial_summary,...,serves_lunch,serves_vegetarian_food,serves_wine,takeout,type,types,url,user_ratings_total,website,wheelchair_accessible_entrance
0,Veg'n Out,"4606 Mission Gorge Pl, San Diego, CA 92120",ChIJpaO4OwdX2YARobj8vAsdzyw,ChIJpaO4OwdX2YARobj8vAsdzyw,OPERATIONAL,False,"[Monday: Closed, Tuesday: Closed, Wednesday: 1...",True,True,NaN,...,True,True,False,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=3228831394100459681,50,http://eatvegnout.com/,True
1,Kenshō,"16511 Magnolia St, Westminster, CA 92683",ChIJmedl7eAn3YAR8fwvMuufkro,ChIJmedl7eAn3YAR8fwvMuufkro,OPERATIONAL,NaN,"[Monday: 12:00 – 9:00 PM, Tuesday: 12:00 – 9:0...",True,True,Plentiful plates of plant-based sushi are on o...,...,True,True,NaN,True,NaN,"[restaurant, food, point_of_interest, establis...",https://maps.google.com/?cid=13443983670162488561,282,http://instagram.com/kenshovegansushi,True
2,Goldilocks Filipino Cuisine (SFO T1),San Francisco International Airport-T1 Harvey ...,ChIJeaxA-QR3j4AR3jYGZt9GO0k,ChIJeaxA-QR3j4AR3jYGZt9GO0k,OPERATIONAL,False,"[Monday: 3:30 AM – 11:30 PM, Tuesday: 3:30 AM ...",False,True,"Down-to-earth pit stop for noodles, rice dishe...",...,True,True,True,True,NaN,"[cafe, bakery, bar, store, restaurant, point_o...",https://maps.google.com/?cid=5276889313698002654,111,http://mamagos.com/,True
3,Veggie Fam,"1500 Griffith Park Blvd, Los Angeles, CA 90189",ChIJXQ9KCkjHwoARDtxNJezKQzs,ChIJXQ9KCkjHwoARDtxNJezKQzs,OPERATIONAL,NaN,"[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",True,True,NaN,...,True,True,NaN,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=4270479987264314382,2,NaN,True
4,Kau Kau,"855 57th St Suite C, Sacramento, CA 95819",ChIJIzLI5A_bmoAR-OgrdVZo5Lg,ChIJIzLI5A_bmoAR-OgrdVZo5Lg,OPERATIONAL,False,"[Monday: Closed, Tuesday: Closed, Wednesday: C...",True,True,NaN,...,True,NaN,True,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=13322888318211057912,98,http://kaukau916.com/,True
5,Lee's Sandwiches,"2471 Berryessa Rd #3, San Jose, CA 95133",ChIJDU7RlUTMj4ARw-Z009ZQ5BE,ChIJDU7RlUTMj4ARw-Z009ZQ5BE,OPERATIONAL,False,"[Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM –...",True,True,Counter-service chain for Vietnamese staples s...,...,True,True,False,True,NaN,"[restaurant, food, point_of_interest, establis...",https://maps.google.com/?cid=1289244276935681731,396,http://leesandwiches.com/,True
6,Lee's Sandwiches,"260 E Santa Clara St, San Jose, CA 95113",ChIJW7ZA0b3Mj4ARYpRDYmu7qso,ChIJW7ZA0b3Mj4ARYpRDYmu7qso,OPERATIONAL,NaN,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",True,True,Counter-service chain for Vietnamese staples s...,...,True,True,NaN,True,NaN,"[restaurant, food, point_of_interest, establis...",https://maps.google.com/?cid=14603690811586745442,745,http://leesandwiches.com/,True
7,Yu Seafood Yorkdale,"3401 Dufferin St Unit 601A, Toronto, ON M6A 2T...",ChIJh5P37QYzK4gRgINfrPNauv8,ChIJh5P37QYzK4gRgINfrPNauv8,OPERATIONAL,False,"[Monday: 10:00 AM – 10:00 PM, Tuesday: 10:00 A...",True,True,NaN,...,True,True,True,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=18427140827955299200,489,https://www.yuseafood.com/yorkdale/,True
8,Bring Me Some,"638 Concession St, Hamilton, ON L8V 1B5, Canada",ChIJddtZ8LebLIgRYTXX7_GjwaU,ChIJddtZ8LebLIgRYTXX7_GjwaU,OPERATIONAL,True,"[Monday: Closed, Tuesday: 11:00 AM – 8:00 PM, ...",True,True,NaN,...,True,True,False,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=11944007946269570401,190,http://www.bringmesome.ca/,True
9,MATHILDA'S - Plant-based eating at its best,"29 Simcoe St S, Oshawa, ON L1H 4G1, Canada",ChIJS7Tuargd1YkRFxl2QJsMSsk,ChIJS7Tuargd1YkRFxl2QJsMSsk,OPERATIONAL,NaN,"[Monday: Closed, Tuesday: 12:00 – 6:00 PM, Wed...",NaN,True,"Laid-back counter-serve doling out pasta, sand...",.

#### Add zipcode column

In [ ]:
zipcodes = pd.Series([addr.split() for addr in merged_df.address]).map(lambda x: x[-1] if x[-1] != 'Canada' else x[-3] + ' ' + x[-2][:-1])
merged_df['zipcode'] = zipcodes

#### Save data to csv

In [ ]:
merged_df.to_csv('data/omni_place_details.tsv', sep='\t')

In [ ]:
merged_df

,name,address,pid,place_id,business_status,curbside_pickup,opening_hours,delivery,dine_in,editorial_summary,...,serves_lunch,serves_vegetarian_food,serves_wine,takeout,type,types,url,user_ratings_total,website,wheelchair_accessible_entrance
0,Veg'n Out,"4606 Mission Gorge Pl, San Diego, CA 92120",ChIJpaO4OwdX2YARobj8vAsdzyw,ChIJpaO4OwdX2YARobj8vAsdzyw,OPERATIONAL,False,"[Monday: Closed, Tuesday: Closed, Wednesday: 1...",True,True,NaN,...,True,True,False,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=3228831394100459681,50,http://eatvegnout.com/,True
1,Kenshō,"16511 Magnolia St, Westminster, CA 92683",ChIJmedl7eAn3YAR8fwvMuufkro,ChIJmedl7eAn3YAR8fwvMuufkro,OPERATIONAL,NaN,"[Monday: 12:00 – 9:00 PM, Tuesday: 12:00 – 9:0...",True,True,Plentiful plates of plant-based sushi are on o...,...,True,True,NaN,True,NaN,"[restaurant, food, point_of_interest, establis...",https://maps.google.com/?cid=13443983670162488561,282,http://instagram.com/kenshovegansushi,True
2,Goldilocks Filipino Cuisine (SFO T1),San Francisco International Airport-T1 Harvey ...,ChIJeaxA-QR3j4AR3jYGZt9GO0k,ChIJeaxA-QR3j4AR3jYGZt9GO0k,OPERATIONAL,False,"[Monday: 3:30 AM – 11:30 PM, Tuesday: 3:30 AM ...",False,True,"Down-to-earth pit stop for noodles, rice dishe...",...,True,True,True,True,NaN,"[cafe, bakery, bar, store, restaurant, point_o...",https://maps.google.com/?cid=5276889313698002654,111,http://mamagos.com/,True
3,Veggie Fam,"1500 Griffith Park Blvd, Los Angeles, CA 90189",ChIJXQ9KCkjHwoARDtxNJezKQzs,ChIJXQ9KCkjHwoARDtxNJezKQzs,OPERATIONAL,NaN,"[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",True,True,NaN,...,True,True,NaN,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=4270479987264314382,2,NaN,True
4,Kau Kau,"855 57th St Suite C, Sacramento, CA 95819",ChIJIzLI5A_bmoAR-OgrdVZo5Lg,ChIJIzLI5A_bmoAR-OgrdVZo5Lg,OPERATIONAL,False,"[Monday: Closed, Tuesday: Closed, Wednesday: C...",True,True,NaN,...,True,NaN,True,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=13322888318211057912,98,http://kaukau916.com/,True
5,Lee's Sandwiches,"2471 Berryessa Rd #3, San Jose, CA 95133",ChIJDU7RlUTMj4ARw-Z009ZQ5BE,ChIJDU7RlUTMj4ARw-Z009ZQ5BE,OPERATIONAL,False,"[Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM –...",True,True,Counter-service chain for Vietnamese staples s...,...,True,True,False,True,NaN,"[restaurant, food, point_of_interest, establis...",https://maps.google.com/?cid=1289244276935681731,396,http://leesandwiches.com/,True
6,Lee's Sandwiches,"260 E Santa Clara St, San Jose, CA 95113",ChIJW7ZA0b3Mj4ARYpRDYmu7qso,ChIJW7ZA0b3Mj4ARYpRDYmu7qso,OPERATIONAL,NaN,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",True,True,Counter-service chain for Vietnamese staples s...,...,True,True,NaN,True,NaN,"[restaurant, food, point_of_interest, establis...",https://maps.google.com/?cid=14603690811586745442,745,http://leesandwiches.com/,True
7,Yu Seafood Yorkdale,"3401 Dufferin St Unit 601A, Toronto, ON M6A 2T...",ChIJh5P37QYzK4gRgINfrPNauv8,ChIJh5P37QYzK4gRgINfrPNauv8,OPERATIONAL,False,"[Monday: 10:00 AM – 10:00 PM, Tuesday: 10:00 A...",True,True,NaN,...,True,True,True,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=18427140827955299200,489,https://www.yuseafood.com/yorkdale/,True
8,Bring Me Some,"638 Concession St, Hamilton, ON L8V 1B5, Canada",ChIJddtZ8LebLIgRYTXX7_GjwaU,ChIJddtZ8LebLIgRYTXX7_GjwaU,OPERATIONAL,True,"[Monday: Closed, Tuesday: 11:00 AM – 8:00 PM, ...",True,True,NaN,...,True,True,False,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=11944007946269570401,190,http://www.bringmesome.ca/,True
9,MATHILDA'S - Plant-based eating at its best,"29 Simcoe St S, Oshawa, ON L1H 4G1, Canada",ChIJS7Tuargd1YkRFxl2QJsMSsk,ChIJS7Tuargd1YkRFxl2QJsMSsk,OPERATIONAL,NaN,"[Monday: Closed, Tuesday: 12:00 – 6:00 PM, Wed...",NaN,True,"Laid-back counter-serve doling out pasta, sand...",.

## Current Foods

### Scrape restaurant Data
get restaurants from current foods

In [95]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://currentfoods.com/find-us/'
driver.get(url)

# wait for searchfield and send "SAN "
search_field = WebDriverWait(driver,10).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, 'storerocket-search-field')))
search_field[0].send_keys('SAN ')

# wait for autocomplete results and press ENTER
WebDriverWait(driver,10).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, 'storerocket-autocomplete-result')))
search_field[0].send_keys(Keys.RETURN)

# wait for availability of searchbox clear button and press it
clear_btn = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'storerocket-clear-search'))).click()

# wait for all possible list results to appear and save their text content in names
names_selector = WebDriverWait(driver, 10).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, 'storerocket-result-content'))
)
names = [name.text.split('\n') for name in names_selector]

# Close the webdriver
driver.quit()

In [309]:
df = pd.DataFrame(names, columns=['name','addr_1','addr_2','country','phone'])
df.head()

,name,addr_1,addr_2,country,phone
0,7Friday Sushi,"18121 TUCKERTON RD, STE 100","CYPRESS, TEXAS 77433",UNITED STATES OF AMERICA,(281) 304-2646
1,Bel Air,1286 STABLER LN,"YUBA CITY, CALIFORNIA 95993",UNITED STATES OF AMERICA,None
2,Bel Air,3250 ARENA BLVD,"SACRAMENTO, CALIFORNIA 95834",UNITED STATES OF AMERICA,None
3,Bel Air,5100 LAGUNA BLVD,"ELK GROVE, CALIFORNIA 95758",UNITED STATES OF AMERICA,None
4,Bel Air,8425 ELK GROVE FLORIN RD,"ELK GROVE, CALIFORNIA 95624",UNITED STATES OF AMERICA,None


### Get place Ids
Populate df with place ids from google Places API

In [125]:
#key api key
with open('/Users/alexanderdaffara/Documents/api_info/ImpactRestaurantSearch/Impact_maps_api_key.txt', 'r') as f:
    key = f.readline()
pids = []
# Iterate over the restaurants and retrieve their Place IDs
for name, address in zip(df.name, df.addr_1):
    # URL for Place Search API
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={name} {address}&inputtype=textquery&fields=place_id&key={key}'

    # Send a GET request to the API endpoint
    response = requests.get(url)

    # Check if the response was successful (HTTP status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()

        # Extract the Place ID from the response
        if data['status'] == 'OK':
            place_id = data['candidates'][0]['place_id']
            pids.append(place_id)
        else:
            print(f'Request for {name} succeeded with status {data["status"]}')
            pids.append(np.nan)
    else:
        # Print an error message if the response was unsuccessful
        print(f'Request for {name} failed with error code {response.status_code}')
        pids.append(np.nan)
df['pid'] = pids

Request for La Sirena - ARGENTONA succeeded with status ZERO_RESULTS
Request for La Sirena - BCN-ANDREA DÒRIA succeeded with status ZERO_RESULTS
Request for La Sirena - CERVERA succeeded with status ZERO_RESULTS
Request for La Sirena - CORNELLÀ-1 succeeded with status ZERO_RESULTS
Request for La Sirena - ESPLUGUES DE LLOBREGAT-1 succeeded with status ZERO_RESULTS
Request for La Sirena - GIRONA-3 succeeded with status ZERO_RESULTS
Request for La Sirena - L'HOSPITALET-3 succeeded with status ZERO_RESULTS
Request for La Sirena - L'HOSPITALET-4 succeeded with status ZERO_RESULTS
Request for La Sirena - L'HOSPITALET-6 succeeded with status ZERO_RESULTS
Request for La Sirena - L'HOSPITALET-7 succeeded with status ZERO_RESULTS
Request for La Sirena - MAD-AMOR DE DIOS succeeded with status ZERO_RESULTS
Request for La Sirena - MAD-JULIÁN ROMEA succeeded with status ZERO_RESULTS
Request for La Sirena - MAD-MAGALLANES succeeded with status ZERO_RESULTS
Request for La Sirena - MAD-MENÉNDEZ PELAYO 

In [308]:
# df with pid's and dropped duplicates and pid nans
df.head()

,name,addr_1,addr_2,country,phone,pid
0,7Friday Sushi,"18121 TUCKERTON RD, STE 100","CYPRESS, TEXAS 77433",UNITED STATES OF AMERICA,(281) 304-2646,ChIJK7_f5VnXQIYR-0Ozp_9oxMA
1,Bel Air,1286 STABLER LN,"YUBA CITY, CALIFORNIA 95993",UNITED STATES OF AMERICA,None,ChIJR-wdqV6rhIARybGPafx54CI
2,Bel Air,3250 ARENA BLVD,"SACRAMENTO, CALIFORNIA 95834",UNITED STATES OF AMERICA,None,ChIJYzyH9wbWmoAR0Etx8rf7k8o
3,Bel Air,5100 LAGUNA BLVD,"ELK GROVE, CALIFORNIA 95758",UNITED STATES OF AMERICA,None,ChIJNWhGYyfGmoARNrnQscNDE3o
4,Bel Air,8425 ELK GROVE FLORIN RD,"ELK GROVE, CALIFORNIA 95624",UNITED STATES OF AMERICA,None,ChIJN7RrprjGmoARgbOv2QruCqU


### Get place details
populate place_details_df with place details corresponding to the retrieved place Ids

In [40]:
# Empty place details
def reset_place_details():
    return {"place_id" : [],
            "business_status" : [],
            "curbside_pickup" : [],
            "opening_hours/weekday_text" : [],
            "delivery"  : [],
            "dine_in" : [],
            "editorial_summary/overview" : [],
            "name" : [],
            "price_level" : [],
            "rating" : [],
            "reservable" : [],
            "serves_beer" : [],
            "serves_breakfast" : [],
            "serves_brunch" : [],
            "serves_dinner" : [],
            "serves_lunch" : [],
            "serves_vegetarian_food" : [],
            "serves_wine" : [],
            "takeout" : [],
            "type" : [],
            "types" : [],
            "url" : [],
            "user_ratings_total" : [],
            "website" : [],
            "wheelchair_accessible_entrance" : [] }

In [41]:
# helper to add row of place details to dictionary
def update_place_details(pdt, data):
    if not data:
        for detail in list(pdt.keys()):
            pdt[detail].append(np.nan)
    else:
        long_details = ["opening_hours/weekday_text", "editorial_summary/overview"]
        details = set(list(pdt.keys())) - set(long_details)
        for detail in details:
            pdt[detail].append(data.get(detail,np.nan))
        for detail in long_details:
            d0, d1 = detail.split('/')
            pdt[detail].append(data.get(d0,{}).get(d1,np.nan))
        
    return pdt

In [249]:
#key api key
with open('/Users/alexanderdaffara/Documents/api_info/ImpactRestaurantSearch/Impact_maps_api_key.txt', 'r') as f:
    key = f.readline()

place_details = reset_place_details()

# Iterate over the restaurants and retrieve their Place IDs
for name, pid in zip(df.name, df.pid):
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={pid}&key={key}"

    response = requests.request("GET", url)

    # Check if the response was successful (HTTP status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()

        # Extract the Place ID from the response
        if data['status'] == 'OK':
            place_details = update_place_details(place_details, data['result'])
        else:
            print(f'Request for {name}, {pid} succeeded with status {data["status"]}')
            place_details = update_place_details(place_details, None)
    else:
        # Print an error message if the response was unsuccessful
        print(f'Request for {name}, {pid} failed with error code {response.status_code}')
        place_details = update_place_details(place_details, None)

Request for Pop Together, ChIJBRk5eXR_j4ARM1axwk9nbpk succeeded with status NOT_FOUND


In [310]:
place_details_df = pd.DataFrame(place_details, columns=place_details.keys())
place_details_df.head()

,place_id,business_status,curbside_pickup,opening_hours/weekday_text,delivery,dine_in,editorial_summary/overview,name,price_level,rating,...,serves_lunch,serves_vegetarian_food,serves_wine,takeout,type,types,url,user_ratings_total,website,wheelchair_accessible_entrance
0,ChIJK7_f5VnXQIYR-0Ozp_9oxMA,OPERATIONAL,True,"[Monday: 7:00 AM – 7:00 PM, Tuesday: 7:00 AM –...",True,True,NaN,7Friday Sushi,NaN,5.0,...,True,True,NaN,True,NaN,"[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=13890342597956944891,136.0,http://www.7fridaysushi.com/,True
1,ChIJR-wdqV6rhIARybGPafx54CI,OPERATIONAL,True,"[Monday: 6:00 AM – 11:00 PM, Tuesday: 6:00 AM ...",True,False,NaN,Bel Air,2.0,4.3,...,NaN,NaN,NaN,True,NaN,"[grocery_or_supermarket, food, store, point_of...",https://maps.google.com/?cid=2513142717082481097,1238.0,https://www.raleys.com/stores/bel-air-stabler-...,True
2,ChIJYzyH9wbWmoAR0Etx8rf7k8o,OPERATIONAL,True,"[Monday: 6:00 AM – 11:00 PM, Tuesday: 6:00 AM ...",True,NaN,NaN,Bel Air,2.0,4.4,...,NaN,NaN,NaN,True,NaN,"[grocery_or_supermarket, store, food, point_of...",https://maps.google.com/?cid=14597287584703007696,1031.0,http://www.raleys.com/,True
3,ChIJNWhGYyfGmoARNrnQscNDE3o,OPERATIONAL,True,"[Monday: 6:00 AM – 11:00 PM, Tuesday: 6:00 AM ...",True,False,NaN,Bel Air,2.0,4.3,...,NaN,NaN,NaN,True,NaN,"[grocery_or_supermarket, food, store, point_of...",https://maps.google.com/?cid=8796449004965640502,690.0,https://www.raleys.com/stores/bel-air-laguna-e...,True
4,ChIJN7RrprjGmoARgbOv2QruCqU,OPERATIONAL,True,"[Monday: 6:00 AM – 11:00 PM, Tuesday: 6:00 AM ...",True,False,NaN,Bel Air,2.0,4.3,...,NaN,NaN,NaN,True,NaN,"[grocery_or_supermarket, food, store, point_of...",https://maps.google.com/?cid=11892579496394470273,1371.0,https://www.raleys.com/stores/bel-air-elk-grov...,True


#### merge Scraping df with Place Details df

In [311]:
merged_df = df.merge(place_details_df.drop(columns='name'),
                     how='inner',
                     left_on='pid',
                     right_on='place_id',
                     suffixes=['_L','_R'])
merged_df.rename(columns={'opening_hours/weekday_text':'opening_hours', 'editorial_summary/overview': 'editorial_summary'}, inplace=True)
merged_df.head()

KeyError: 'pid'

In [ ]:
# add zipcode column
zipcodes = [addr.split()[-1] for addr in merged_df.addr_2]
merged_df['zipcode'] = zipcodes

# # manual zipcode input for one row
merged_df.loc[df.pid == 'ChIJ95l0SeR7pBIRGysUsgq0HX0','zipcode'] = '08799'

#### Save data to csv

In [306]:
merged_df.to_csv('data/place_details.tsv', sep='\t')

# CF Extra Info

In [7]:
rest_df = pd.read_csv('data/cf_restaurants.tsv', sep='\t', index_col=0)
rest_df

,name,url,addr_1,addr_2,country,curbside_pickup,delivery,dine_in,price_level,rating,...,serves_lunch,serves_vegetarian_food,serves_wine,takeout,user_ratings_total,wheelchair_accessible_entrance,zipcode,editorial_summary,types,categories
0,7Friday Sushi,https://maps.google.com/?cid=13890342597956944891,"18121 TUCKERTON RD, STE 100","CYPRESS, TEXAS 77433",UNITED STATES OF AMERICA,True,True,True,NaN,5.0,...,True,True,NaN,True,136.0,True,77433,NaN,"['restaurant', 'point_of_interest', 'food', 'e...",NaN
21,Bestie's Vegan Paradise,https://maps.google.com/?cid=16874039323387828855,4882 FOUNTAIN AVE,"LOS ANGELES, CALIFORNIA 90029",UNITED STATES OF AMERICA,True,True,NaN,NaN,4.5,...,NaN,NaN,NaN,True,225.0,True,90029,NaN,"['grocery_or_supermarket', 'meal_takeaway', 'r...",NaN
22,blue pokē,https://maps.google.com/?cid=13134429803069602710,"1325 E PLAZA BLVD, STE 103","NATIONAL CITY, CALIFORNIA 91950",UNITED STATES OF AMERICA,True,True,True,2.0,4.6,...,True,True,False,True,594.0,True,91950,NaN,"['restaurant', 'point_of_interest', 'food', 'e...",NaN
23,Blue Sushi Sake Grill - Power and Light,https://maps.google.com/?cid=6359419975810682658,101 E 14TH ST,"KANSAS CITY, MISSOURI 64106",UNITED STATES OF AMERICA,NaN,True,True,2.0,4.9,...,True,True,True,True,37.0,True,64106,"Modern, clubby restaurant featuring sushi & Ja...","['restaurant', 'point_of_interest', 'food', 'e...","Sushi Bars, Restaurants, Nightlife, Cocktail B..."
24,Blue Sushi Sake Grill - Court Avenue,https://maps.google.com/?cid=8663256725197864751,316 COURT AVE,"DES MOINES, IOWA 50309",UNITED STATES OF AMERICA,NaN,True,True,2.0,4.5,...,True,True,True,True,656.0,True,50309,"Modern, clubby restaurant featuring sushi & Ja...","['restaurant', 'food', 'point_of_interest', 'e...","Sushi Bars, Restaurants, Nightlife, Cocktail B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Sancha Sushi,https://maps.google.com/?cid=17639356127951289174,1224 GRANT AVE,"SAN FRANCISCO, CALIFORNIA 94133",UNITED STATES OF AMERICA,NaN,True,True,NaN,4.4,...,True,NaN,NaN,True,48.0,True,94133,NaN,"['restaurant', 'point_of_interest', 'food', 'e...",NaN
503,Sancha Sushi,https://maps.google.com/?cid=1468585188584661767,253 CHURCH ST,"SAN FRANCISCO, CALIFORNIA 94114",UNITED STATES OF AMERICA,NaN,True,True,2.0,4.8,...,True,NaN,True,True,53.0,True,94114,NaN,"['restaurant', 'point_of_interest', 'food', 'e...",NaN
504,SeaDog Sushi Bar,https://maps.google.com/?cid=11997228980446377630,"1500 W DIVISION ST, #3344","CHICAGO, ILLINOIS 60642",UNITED STATES OF AMERICA,False,True,True,2.0,4.3,...,True,True,True,True,324.0,NaN,60642,Brick-lined Japanese BYOB spot with a modern m...,"['veterinary_care', 'meal_takeaway', 'restaura...",NaN
506,Trader Vic's - Emeryville,https://maps.google.com/?cid=16606159308439485126,9 ANCHOR DR,"EMERYVILLE, CALIFORNIA 94608",UNITED STATES OF AMERICA,True,False,True,3.0,4.4,...,True,True,True,True,1139.0,True,94608,Festive Polynesian-style fixture boasts creati...,"['bar', 'restaurant', 'food', 'point_of_intere...",NaN


### Scrape GMaps for extra info

In [121]:
data = []
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(rest_df.url.iloc[98])
# wait for options and select restaurants
WebDriverWait(driver,3).until(
    EC.visibility_of_element_located((By.CLASS_NAME, "y0K5Df"))).click() 
text= []
n = len(WebDriverWait(driver,3).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "iP2t7d"))))
for i in range(n):
    text.append(WebDriverWait(driver,3).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "iP2t7d")))[i].text)
this_data = {key: val for key, val in zip([s.split('\n')[0] for s in text], [s.split('\n')[1:] for s in text])}
data.append(this_data)
time.sleep(10)
print(f'{i}/99')
# Close the webdriver
driver.quit()

10/99


In [99]:
d_is = []
counter = 33
while counter < 99:
    data = []
    try:
        for i, url in enumerate(rest_df.url[counter:]):
            driver = webdriver.Chrome(ChromeDriverManager().install())

            driver.get(url)
            # wait for options and select restaurants
            WebDriverWait(driver,3).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "y0K5Df"))).click() 
            text= []
            n = len(WebDriverWait(driver,3).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "iP2t7d"))))
            for i in range(n):
                text.append(WebDriverWait(driver,3).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "iP2t7d")))[i].text)
            this_data = {key: val for key, val in zip([s.split('\n')[0] for s in text], [s.split('\n')[1:] for s in text])}
            data.append(this_data)
            time.sleep(10)
            print(f'{i}/99')
        counter = 99
        break
    except:
        i = len(data)
        if i == 0: 
            print("no new data")
            break
        d_i = pd.DataFrame(data)
        d_is.append(d_i)
        counter += i
        print(f'{i} new data points')
    finally:
        # Close the webdriver
        driver.quit()
        continue


10/99
8/99
2 new data points
8/99
8/99
8/99
3 new data points
8/99
1 new data points
no new data
8/99
1 new data points
no new data
no new data
8/99
1 new data points
8/99
8/99
8/99
8/99
4 new data points
8/99
1 new data points
no new data
8/99
8/99
2 new data points
8/99
1 new data points
8/99
8/99
2 new data points
8/99
1 new data points
no new data
8/99
8/99
8/99
8/99
4 new data points
8/99
1 new data points
no new data
7/99
1 new data points
no new data
8/99
8/99
8/99
8/99
8/99
8/99
8/99
7 new data points
8/99
8/99
2 new data points
no new data
no new data
no new data
8/99
1 new data points
no new data
no new data
no new data
no new data
no new data
no new data
no new data
no new data
8/99
8/99
8/99
3 new data points
8/99
1 new data points
8/99
8/99
2 new data points
no new data
no new data
8/99
8/99
8/99
3 new data points
8/99
8/99
8/99
3 new data points
no new data
no new data
no new data
8/99
1 new data points
no new data
no new data
no new data
8/99
8/99
8/99
3 new data points


In [124]:
final_google_df = pd.concat(*d_is).reset_index(inplace=True, drop=True)
final_google_df.to_csv('data/cf_restaurants_extra_data.tsv', sep='\t')

,Service options,Popular for,Accessibility,Offerings,Dining options,Amenities,Atmosphere,Planning,Payments,Crowd,Recycling,Highlights,From the business
0,"[Curbside pickup, Drive-through, Takeout, Dine...","[Lunch, Dinner, Solo dining]","[Wheelchair accessible entrance, Wheelchair ac...","[Healthy options, Quick bite, Small plates, Ve...","[Breakfast, Lunch, Dinner, Dessert]","[Good for kids, Restroom]","[Casual, Cozy]",[Accepts reservations],"[Debit cards, NFC mobile payments, Credit cards]",NaN,NaN,NaN,NaN
1,"[Curbside pickup, No-contact delivery, Deliver...",NaN,"[Wheelchair accessible entrance, Wheelchair ac...","[Organic products, Prepared foods]",NaN,[Free Wi-Fi],NaN,[Quick visit],"[Debit cards, NFC mobile payments, SNAP/EBT, C...","[LGBTQ+ friendly, Transgender safespace]","[Glass bottles, Metal cans]",NaN,NaN
2,"[Curbside pickup, No-contact delivery, Deliver...","[Lunch, Dinner, Solo dining]","[Wheelchair accessible entrance, Wheelchair ac...","[Comfort food, Healthy options, Organic dishes...","[Lunch, Dinner, Catering, Dessert, Seating]","[Good for kids, High chairs, Restroom]",[Casual],NaN,"[Debit cards, NFC mobile payments, Credit cards]","[College students, Family-friendly, Groups, LG...",NaN,NaN,NaN
3,"[Delivery, Takeout, Dine-in]","[Lunch, Dinner, Solo dining]","[Wheelchair accessible entrance, Wheelchair ac...","[Alcohol, Beer, Cocktails, Happy hour drinks, ...","[Lunch, Dinner, Catering, Dessert, Seating]",NaN,[Casual],[Accepts reservations],"[Debit cards, NFC mobile payments, Credit cards]",NaN,NaN,[Fast service],NaN
4,"[Outdoor seating, Delivery, Takeout, Dine-in]","[Lunch, Dinner, Solo dining]","[Wheelchair accessible entrance, Wheelchair ac...","[Alcohol, Beer, Cocktails, Happy hour drinks, ...","[Lunch, Dinner, Catering, Dessert, Seating]","[Bar onsite, High chairs, Restroom]","[Casual, Cozy, Romantic, Upscale]",[Accepts reservations],"[Debit cards, NFC mobile payments, Credit cards]","[Groups, Tourists]",NaN,"[Fast service, Great tea selection]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,"[Delivery, Takeout, Dine-in]",NaN,[Wheelchair accessible seating],"[Alcohol, Beer, Cocktails, Coffee, Hard liquor...","[Breakfast, Lunch, Dessert]","[Bar onsite, Dogs allowed, Restroom]",[Casual],[Accepts reservations],NaN,NaN,NaN,NaN,[Identifies as women-owned]
2,"[Delivery, Takeout, Dine-in]","[Lunch, Dinner, Solo dining]","[Wheelchair accessible entrance, Wheelchair ac...","[Healthy options, Quick bite, Small plates]","[Lunch, Dinner]",NaN,[Casual],NaN,NaN,NaN,NaN,NaN,NaN
0,"[Delivery, Takeout, Dine-in]","[Lunch, Dinner, Solo dining]","[Wheelchair accessible entrance, Wheelchair ac...","[Alcohol, Beer, Healthy options, Small plates,...","[Lunch, Dinner]",[Restroom],[Casual],[Accepts reservations],"[Debit cards, NFC mobile payments, Credit cards]",NaN,NaN,[Fast service],NaN
0,"[Outdoor seating, No-contact delivery, Deliver...","[Lunch, Dinner, Solo dining]","[Wheelchair accessible restroom, Wheelchair ac...","[Healthy options, Vegan options, Vegetarian op...","[Lunch, Dinner, Catering, Dessert, Seating]","[High chairs, Restroom]","[Casual, Cozy, Romantic]","[Dinner reservations recommended, Accepts rese...","[Debit cards, Credit cards]","[Family-friendly, Groups]",NaN,[Great tea selection],NaN
